In [26]:
# %pip install llama_index
# %pip install llama-index-embeddings-huggingface
# %pip install llama-index-llms-ollama


from llama_index.core import VectorStoreIndex, SimpleDirectoryReader, Settings
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
from llama_index.llms.ollama import Ollama

from llama_index.agent.openai import OpenAIAgent
from llama_index.core import load_index_from_storage, StorageContext
from llama_index.core.node_parser import SentenceSplitter
import os
from llama_index.core import SummaryIndex
from llama_index.core.tools import QueryEngineTool, ToolMetadata
from llama_index.core.agent import ReActAgent
from llama_index.core.tools.query_engine import QueryEngineTool
from llama_index.core.query_engine import RouterQueryEngine


In [27]:
import torch
# Verificar si la GPU está disponible
if torch.cuda.is_available():
    device = torch.device("cuda")
    print("GPU is available")
else:
    device = torch.device("cpu")
    print("GPU is not available, using CPU")

device = torch.device("cuda")


GPU is available


In [28]:
import os

# Crear una lista con los nombres de los ficheros en el directorio "transcripciones"
directory_path = "C:/Users/gmsol/Desktop/videodescargas/transcripciones"


In [29]:


# bge-base embedding model
Settings.embed_model = HuggingFaceEmbedding(model_name="BAAI/bge-base-en-v1.5")

# ollama
# importante: installar ollama y levantar el modelo que quieras usar por linea de comandos (https://github.com/ollama/ollama?tab=readme-ov-file)
Settings.llm = Ollama(model="llama3.1", request_timeout=360.0, device_map=device, temperature=0.1)
# Settings.llm.to(device)

In [31]:
Settings.llm.model

'llama3.1'

In [32]:
# Obtener una lista con los nombres de los ficheros en el directorio "transcripciones"
file_list = os.listdir(directory_path)
print(file_list)

['CmoGanarCualquierDiscusinFerMirallestxt', 'CmoPodra1TrillndeLeonesGanarlealSoltxt', 'Intel5000ExtremeTechUpgradetxt', 'UnaClaseconelDrdelSueotxt']


In [51]:


# Función para crear un Query Engine a partir de un fichero
def create_qe(file_path):
    # Cargar el documento desde un archivo individual
    documents = SimpleDirectoryReader(input_files=[file_path]).load_data()
    index = VectorStoreIndex.from_documents(documents)
    return index.as_query_engine()



# Crear una lista para almacenar las herramientas
query_engine_tools = []

# Iterar sobre los archivos .txt en el directorio
for file_name in os.listdir(directory_path):
    print(f"Procesando archivo: {file_name}")
    file_path = os.path.join(directory_path, file_name)
    print(file_path)
    
    # Verificar si es un archivo .txt
    
    print(f"Procesando archivo: {file_path}")
        
        # Crear el Query Engine para el archivo
    query_engine = create_qe(file_path)
        
        # Crear el QueryEngineTool para este archivo
    query_engine_tool = QueryEngineTool.from_defaults(
            query_engine=query_engine,
            description=f"Útil cuando se hacen preguntas sobre {file_name}"
        )
        
        # Agregar la herramienta a la lista
    query_engine_tools.append(query_engine_tool)

# Crear el RouterQueryEngine con todas las herramientas creadas dinámicamente
router_qe = RouterQueryEngine.from_defaults(
    query_engine_tools=query_engine_tools
)


Procesando archivo: CmoGanarCualquierDiscusinFerMirallestxt
C:/Users/gmsol/Desktop/videodescargas/transcripciones\CmoGanarCualquierDiscusinFerMirallestxt
Procesando archivo: C:/Users/gmsol/Desktop/videodescargas/transcripciones\CmoGanarCualquierDiscusinFerMirallestxt
Procesando archivo: CmoPodra1TrillndeLeonesGanarlealSoltxt
C:/Users/gmsol/Desktop/videodescargas/transcripciones\CmoPodra1TrillndeLeonesGanarlealSoltxt
Procesando archivo: C:/Users/gmsol/Desktop/videodescargas/transcripciones\CmoPodra1TrillndeLeonesGanarlealSoltxt
Procesando archivo: Intel5000ExtremeTechUpgradetxt
C:/Users/gmsol/Desktop/videodescargas/transcripciones\Intel5000ExtremeTechUpgradetxt
Procesando archivo: C:/Users/gmsol/Desktop/videodescargas/transcripciones\Intel5000ExtremeTechUpgradetxt
Procesando archivo: UnaClaseconelDrdelSueotxt
C:/Users/gmsol/Desktop/videodescargas/transcripciones\UnaClaseconelDrdelSueotxt
Procesando archivo: C:/Users/gmsol/Desktop/videodescargas/transcripciones\UnaClaseconelDrdelSueotxt


In [40]:
import nest_asyncio

nest_asyncio.apply()

In [65]:
question = "Quien ganaria, los leones o el sol?"
prompt =f'''Tienes acceso a transcripciones de videos de youtube.
            Se te hará una pregunta y debes responder basándote exclusivamente en la información proporcionada en las transcripciones. Si no puedes responder la pregunta utilizando estos datos, indica que la información no está disponible.

            Responde de manera educada, estructurada y concisa cuando corresponda. Si la pregunta requiere una respuesta breve, sé claro y directo. Asegúrate de:

            No inventar información adicional ni suponer nada que no esté claramente expresado en las transcripciones.
            Responder solo utilizando la información proporcionada, sin hacer inferencias más allá del contenido.
            Estructurar la respuesta de manera clara y fácil de entender.
            Mantener un tono respetuoso y profesional en todo momento.
            
            PREGUNTA:
            {question}
            '''
response = router_qe.query(question)
print(response)



El Sol es una bola con un radio de 700.000 kilómetros y una masa de 2 quintillones de kilos, mientras que un trillón de leones formaría una esfera con un radio de unos 400 km y un peso de 200 trillones de kilos. A pesar del tamaño considerable de la bola de leones, su energía sería insuficiente para destruir el Sol, incluso si se lanzara a una velocidad cercana a la velocidad de la luz. Por lo tanto, es probable que el Sol ganara en esta batalla.


In [64]:
response.source_nodes[0].metadata

{'file_path': 'C:\\Users\\gmsol\\Desktop\\videodescargas\\transcripciones\\CmoPodra1TrillndeLeonesGanarlealSoltxt',
 'file_name': 'CmoPodra1TrillndeLeonesGanarlealSoltxt',
 'file_size': 11994,
 'creation_date': '2024-09-13',
 'last_modified_date': '2024-09-13'}